In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd 
import numpy as np 

from xgboost import XGBClassifier,DMatrix,train
from xgboost.callback import EarlyStopping
from xgboost import cv

from sklearn.metrics import f1_score,confusion_matrix,roc_auc_score,recall_score,accuracy_score,precision_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.utils import resample

In [3]:
X_train=np.array(pd.read_csv('C:\workspace\Credit-Risk-Modeling\dataset\kaggle_pd_estimation\X_train.csv',index_col=0))
y_train=np.array(pd.read_csv('C:\workspace\Credit-Risk-Modeling\dataset\kaggle_pd_estimation\y_train.csv',index_col=0).astype(int)).reshape(-1)

X_test=np.array(pd.read_csv('C:\workspace\Credit-Risk-Modeling\dataset\kaggle_pd_estimation\X_test.csv',index_col=0))
y_test=np.array(pd.read_csv('C:\workspace\Credit-Risk-Modeling\dataset\kaggle_pd_estimation\y_test.csv',index_col=0).astype(int)).reshape(-1).astype(int)


In [4]:
train_np=np.concatenate((X_train,y_train.reshape(-1,1)),axis=1)

In [5]:
param={'n_estimators':list(np.linspace(10,2000,20)),'max_depth':[3,5,8,12,15],'eta':list(np.linspace(0.05,0.2,10)),'objective':'binary:logistic','tree_method':'hist'}

In [6]:
def homeMade_f1(y_true,y_pred):
    y_true=pd.Series(y_true).astype(int)
    y_pred=pd.Series(y_pred).astype(int)
    f1_acc_score=f1_score(y_true,y_pred)
    return f1_acc_score

In [7]:
def make_balance(fit_df):
    fit_df_0=fit_df[fit_df[:,-1]==0]
    fit_df_1=fit_df[fit_df[:,-1]==1]
    max_count=np.floor(len(fit_df_0)/len(fit_df_1))
    residuals=int(len(fit_df_0)-max_count*len(fit_df_1))
    balance_count=1
    fit_df2=np.array(pd.DataFrame(fit_df_1).copy())
    while balance_count<max_count:
        fit_df2=np.concatenate((fit_df2,fit_df_1))
        balance_count=balance_count+1
    fit_df2=np.concatenate((fit_df2,resample(fit_df_1,replace=False,n_samples=residuals)))
    fit_df2=np.concatenate((fit_df2,fit_df_0))
    fit_df2=resample(fit_df2,replace=False,n_samples=len(fit_df2))
    return fit_df2

In [44]:
def ensemble_xgb(train_np,params,test_size=0.3):
    n_estimators=params['n_estimators']
    object=params['objective']
    tree_method=params['tree_method']
    depth_list=params['max_depth']
    eta_list=params['eta']
    model_dic={}
    model_number=1
    
    fit_df,val_df=train_test_split(train_np,random_state=None,test_size=test_size,stratify=train_np[:,-1])
    
    fit_df2=make_balance(fit_df) 
    X_fit=fit_df2[:,:-1]
    y_fit=fit_df2[:,-1]

    val_df2=make_balance(val_df) 
    X_val=val_df2[:,:-1]
    y_val=val_df2[:,-1]

    for i in depth_list:
        for j in eta_list:
            for k in n_estimators:
                model=XGBClassifier(n_estimators=np.round(k),objective=object,tree_method=tree_method,early_stopping_rounds=500000,
                                    learning_rate=j,max_depth=i,n_jobs=-1,eval_metric=['logloss']).fit(X_fit,y_fit,eval_set=[(X_val,y_val)])
                #f_1_accuracy_score=f1_score(y_val,pred)
                model_dic['model_number_{}'.format(model_number)]=\
                    {'n_estimators':model.best_iteration,'max_depth':i,'learning_rate':j,'accuracy_score':model.best_score}
                print(model_dic['model_number_{}'.format(model_number)])
                model_number=model_number+1


    model_df=pd.DataFrame(model_dic)
    
    return model_df


In [45]:
param1={'n_estimators':[2000],'max_depth':[3,4,5,6,7,8],'eta':[0.1,0.09,0.08,0.07,0.06,0.05],'objective':'binary:logistic','tree_method':'hist'}
len(param1['n_estimators'])*len(param1['max_depth'])*len(param1['eta'])

36

In [46]:
selected_model1=ensemble_xgb(train_np,param1)

[0]	validation_0-logloss:0.65182
[1]	validation_0-logloss:0.61794
[2]	validation_0-logloss:0.59025
[3]	validation_0-logloss:0.56708
[4]	validation_0-logloss:0.54711
[5]	validation_0-logloss:0.53013
[6]	validation_0-logloss:0.51501
[7]	validation_0-logloss:0.50226
[8]	validation_0-logloss:0.49140
[9]	validation_0-logloss:0.48191
[10]	validation_0-logloss:0.47344
[11]	validation_0-logloss:0.46563
[12]	validation_0-logloss:0.45848
[13]	validation_0-logloss:0.45276
[14]	validation_0-logloss:0.44619
[15]	validation_0-logloss:0.44085
[16]	validation_0-logloss:0.43653
[17]	validation_0-logloss:0.43244
[18]	validation_0-logloss:0.42796
[19]	validation_0-logloss:0.42486
[20]	validation_0-logloss:0.42052
[21]	validation_0-logloss:0.41710
[22]	validation_0-logloss:0.41402
[23]	validation_0-logloss:0.41108
[24]	validation_0-logloss:0.40810
[25]	validation_0-logloss:0.40544
[26]	validation_0-logloss:0.40298
[27]	validation_0-logloss:0.40036
[28]	validation_0-logloss:0.39727
[29]	validation_0-loglos

In [ ]:
selected_model1=selected_model1.transpose().sort_values(['accuracy_score','learning_rate','max_depth'],ascending=[False,False,True])

In [33]:
selected_model1

,n_estimators,max_depth,learning_rate,accuracy_score
model_number_19,262.0,6.0,0.10,0.959694
model_number_15,528.0,5.0,0.08,0.959454
model_number_24,548.0,6.0,0.05,0.959398
model_number_21,413.0,6.0,0.08,0.959394
model_number_14,352.0,5.0,0.09,0.959272
model_number_20,300.0,6.0,0.09,0.959229
model_number_30,398.0,7.0,0.05,0.959083
model_number_17,643.0,5.0,0.06,0.959039
model_number_11,768.0,4.0,0.06,0.958964
model_number_23,441.0,6.0,0.06,0.958859


In [34]:
train_np2=make_balance(train_np) 
X_train=train_np2[:,:-1]
y_train=train_np2[:,-1]
model=XGBClassifier(n_estimators=262,max_depth=6,learning_rate=0.1,n_jobs=-1,objective='binary:logistic').fit(X_train,y_train)
pred=model.predict(X_test)

In [35]:
confusion_matrix(y_test,pred)

array([[4889,  206],
       [ 292, 1130]], dtype=int64)

In [36]:
confusion_matrix(y_test,pred)

array([[4889,  206],
       [ 292, 1130]], dtype=int64)

In [37]:
recall_score(y_test,pred)

0.7946554149085795

In [38]:
precision_score(y_test,pred)

0.8458083832335329

In [39]:
f1_score(y_test,pred)

0.8194343727338651

In [40]:
accuracy_score(y_test,pred)

0.923584471382538